# TensorFlow 共享变量
训练神经网络中，使用共享变量的好处：
1. 减少训练参数的个数
2. 多机并行化的情况下，对变量名和操作名的管理
3. 维护逻辑清晰的计算图（Graph）

共享变量的操作：
* tf.Variable()
* tf.get_variable()
* tf.Variable_scope()
* tf.name_scope()

In [1]:
import numpy as np
import tensorflow as tf

config = tf.ConfigProto(log_device_placement=True)
sess = tf.Session(config=config)

### ```tf.Variable(name=<name>)``` 用来创建一个新变量，变量在同一个 name_scope 下面，如果重名，底层会自动引入别名
var1/var2/var3/var4 都是用 tf.Variable() 创建，并且名字都是 var，得到的结果：var1/var2 都属于默认的 scope，var3/var4 都属于 name_scope_0 的 scope，而在同一个 scope 中，重名的变量被自动的重命名，加上了后缀 ```_1```

In [2]:
var1 = tf.Variable(name='var', initial_value=[2], dtype=tf.float32)
var2 = tf.Variable(name='var', initial_value=[2], dtype=tf.float32)

with tf.name_scope('name_scope_0'):
    var3 = tf.Variable(name='var', initial_value=[2], dtype=tf.float32)
    var4 = tf.Variable(name='var', initial_value=[2], dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(var1.name, sess.run(var1))
    print(var2.name, sess.run(var2))
    print(var3.name, sess.run(var3))
    print(var4.name, sess.run(var4))

var:0 [ 2.]
var_1:0 [ 2.]
name_scope_0/var:0 [ 2.]
name_scope_0/var_1:0 [ 2.]


### ```tf.get_variable(name=<name>)``` 用来获取一个变量，如果没有则创建。并且该方法不受 name_scope() 的影响，同时在没有设置使用共享变量的情况下，会报错。
1. var1 和 var2 都是在一个默认的 scope 下，并不会受 name_scope() 影响
2. var1 和 var2 重名，所以会报错

In [3]:
var1 = tf.get_variable(name='var1', shape=[2], dtype=tf.float32)
with tf.name_scope('name_scope_0'):
    var2 = tf.get_variable(name='var2', shape=[2], dtype=tf.float32)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(var1.name, sess.run(var1))
    print(var2.name, sess.run(var2))

var1:0 [ 0.43267941  1.17510712]
var2:0 [-0.92306077 -1.13661373]


In [4]:
var1 = tf.get_variable(name='var', shape=[2], dtype=tf.float32)
with tf.name_scope('name_scope_0'):
    # var2 = tf.get_variable(name='var', shape=[2], dtype=tf.float32) 该行会报错pass
    pass
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(var1.name, sess.run(var1))
    # print(var2.name, sess.run(var2))

# ERROR
# ValueError: Variable var already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

var_2:0 [-1.0867722   0.93245447]


### ```tf.name_scope(<score_name>)``` 主要用于管理一个计算图中的各种操作（op），返回一个以 scope_name 命名的 context manager
### ```tf.variable_scope(<scope_name>)``` 与 ```tf.name_scope(<score_name>)``` 配合使用，用于管理 scope 中的变量名，避免冲突
* 使用 ```scope.reuse_variables()``` 设置之后，就不会报错了。

In [5]:
with tf.variable_scope('name_scope_1') as scope:
    var1 = tf.get_variable(name='var1', shape=[1], dtype=tf.float32)
    scope.reuse_variables()  # 设置共享变量
    var2 = tf.get_variable(name='var1')
    var3 = tf.Variable(initial_value=[2.], name='var2', dtype=tf.float32)
    var4 = tf.Variable(initial_value=[2.], name='var2', dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(var1.name, sess.run(var1))
    print(var2.name, sess.run(var2))
    print(var3.name, sess.run(var3))
    print(var4.name, sess.run(var4))

name_scope_1/var1:0 [-1.34448731]
name_scope_1/var1:0 [-1.34448731]
name_scope_1/var2:0 [ 2.]
name_scope_1/var2_1:0 [ 2.]
